In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import fsspec
import h5py
import os

import numpy as np
import pandas as pd
import xarray as xr

import nsidc_download_glas as glas_download
from glas_data import *

import matplotlib.pyplot as plt

In [5]:
data14 = xr.open_zarr('../data/glas/washington/GLAH14.zarr')
data01 = xr.open_zarr('../data/glas/washington/GLAH01.zarr')

combined = data14.merge(data01)

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 55 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 93 times more chunks
  return self.array[key]
/srv/conda

In [6]:
combined

<xarray.Dataset>
Dimensions:                   (rec_bin: 544, record_index: 25307, shot_number: 40, tx_bin: 48)
Coordinates:
  * record_index              (record_index) int64 22232240 ... 1069303896
  * shot_number               (shot_number) int64 1 2 3 4 5 6 ... 36 37 38 39 40
  * rec_bin                   (rec_bin) int64 0 1 2 3 4 ... 539 540 541 542 543
  * tx_bin                    (tx_bin) int64 0 1 2 3 4 5 6 ... 42 43 44 45 46 47
Data variables:
    centroid_offset           (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    elevation                 (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    elevation_correction      (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    lat                       (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    lon                       (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    ref_range                 (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    sig_begin_offset          (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    sig_end_offset            (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    time                      (record_index, shot_number) float64 dask.array<chunksize=(6327, 10), meta=np.ndarray>
    noise_mean                (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    noise_sd                  (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf                    (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 44, 5), meta=np.ndarray>
    rec_wf_location_ind       (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf_response_end_time  (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    rec_wf_sample_loc         (rec_bin, record_index, shot_number) float64 dask.array<chunksize=(34, 44, 5), meta=np.ndarray>
    tx_wf                     (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 44, 10), meta=np.ndarray>
    tx_wf_peak_time           (record_index, shot_number) float64 dask.array<chunksize=(44, 10), meta=np.ndarray>
    tx_wf_sample_loc          (tx_bin, record_index, shot_number) float64 dask.array<chunksize=(6, 44, 10), meta=np.ndarray>

In [7]:
SPEED_OF_LIGHT = 299792458   # m/s
SECONDS_IN_NANOSECONDS = 10 ** -9 

def calculate_derived_variables(ds):
    """
    Calculate derived variables in a xarray dataset containing glas data 
    """
    # convert receiving waveform digital bins from 0-543 to corresponding distance from satellite 
    ds['rec_wf_sample_distance'] = ((
        ds.rec_wf_sample_loc
        + ds.rec_wf_response_end_time
        - ds.tx_wf_peak_time
    ) * SECONDS_IN_NANOSECONDS * SPEED_OF_LIGHT) / 2

    # calculate the bias between reference range to the bottom of received wf 
    ds['ref_range_bias'] = ds.rec_wf_sample_distance.max(dim='rec_bin') - ds.ref_range

    # convert offsets to distance from satellite 
    ds['sig_begin_distance'] = ds.sig_begin_offset + ds.ref_range + ds.ref_range_bias
    ds['sig_end_distance'] = ds.sig_end_offset + ds.ref_range + ds.ref_range_bias
    ds['centroid_distance'] = ds.centroid_offset + ds.ref_range + ds.ref_range_bias

    return ds

In [53]:
from scipy import optimize
from scipy.ndimage import gaussian_filter1d

def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-1 * ((x - mean)**2) / (2 * (stddev**2)))

def find_gaussian_fit_sigma(wf):
    """
    wf is a 1D array, not a vectorized function
    """
    x = np.arange(len(wf))
    y = wf - wf.min()  # optimizezr can't deal with negative number 
    popt, _ = optimize.curve_fit(gaussian, x, y, p0 = [0.5, 25, 1])
    
    return popt[2]

def smooth_wf(rec_wf, tx_wf):
    """
    Find sigma from transmitted waveform, and apply gaussian filter smoothing on the recieved waveform with said sigma
    """
    sigma = find_gaussian_fit_sigma(tx_wf)
    
    return gaussian_filter1d(input=rec_wf, sigma=sigma)

def preprocess_wf(ds):
    """
    Smooth and de-noise received waveform, input is an xarray dataset with rec_wf, tx_wf, and noise_mean as dataarrays. 
    Output is a dataarray containing the processed received waveform 
    """
    smoothed_wf = xr.apply_ufunc(
        smooth_wf,
        ds.rec_wf,
        ds.tx_wf,
        input_core_dims=[['rec_bin'], ['tx_bin']],
        output_core_dims=[['rec_bin']],
        vectorize=True,
        dask='parallelized',
        dask_gufunc_kwargs={'allow_rechunk': 1},
        output_dtypes=np.float64
    )
    
    denoised = smoothed_wf - ds.noise_mean 
    
    return denoised
    
def main(ds):
    ds = calculate_derived_variables(ds)
    ds['processed_wf'] = preprocess_wf(ds)

In [9]:
combined = calculate_derived_variables(combined)

In [21]:
sub = combined.isel(record_index=slice(3))

In [47]:
a = xr.apply_ufunc(
    smooth_wf,
    sub.rec_wf,
    sub.tx_wf,
    input_core_dims=[['rec_bin'], ['tx_bin']],
    output_core_dims=[['rec_bin']],
    vectorize=True,
    dask='parallelized',
    dask_gufunc_kwargs={'allow_rechunk': 1},
    output_dtypes=np.float64
).compute()

In [39]:
sub.rec_wf.values.shape

(544, 3, 40)

In [38]:
a.values.shape

(3, 40, 544)

In [30]:
# TODO
# 1) find all the unique rec_wf_sample_loc forms, and make sure that if the compression ratio changed, it changed before signal beginning distance 
# 2) 

<xarray.DataArray (record_index: 25307, shot_number: 40)>
dask.array<sub, shape=(25307, 40), dtype=float64, chunksize=(180, 5), chunktype=numpy.ndarray>
Coordinates:
  * record_index  (record_index) int64 22232240 22232245 ... 1069303896
  * shot_number   (shot_number) int64 1 2 3 4 5 6 7 8 ... 34 35 36 37 38 39 40

In [9]:
# for each record/shot, get the width of tx wf (in ns), create the smoothed waveform

In [ ]:
    # move the reference range to the bottom of received wf 
    bias = bins.max() - other_data.ref_range.iloc[0]
    sig_beg = other_data.sig_begin_distance.iloc[0] + bias
    sig_end = other_data.sig_end_distance.iloc[0] + bias